In [1]:
import rospy
from human_moveit_config.human_model import HumanModel
from sensor_msgs.msg import JointState
from os.path import join
import json
from rospkg import RosPack
from baxter_commander.persistence import dicttostate
import re

import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd
import numpy as np

import socket
import binascii
import struct

In [2]:
rospy.init_node('testFeedback')

In [3]:
rospack = RosPack()
postures_file = join(rospack.get_path('reba_optim'), 'tmp', 'postures.json')
with open(postures_file) as datafile:
    postures_data = json.load(datafile)

In [4]:
pub = rospy.Publisher('reba_assess', JointState, queue_size=10)

In [5]:
human = HumanModel(control=True)

In [6]:
rate = rospy.Rate(5)

In [7]:
####### SINGLE STATE TEST #######
nb_states = 1
state = human.get_random_state()
human.send_state(state)
for i in range(nb_states):
    pub.publish(state)
    rate.sleep()

In [ ]:
####### RANDOM STATE TEST #######
for i in range(10000):
    state = human.get_random_state()
    human.send_state(state)
    pub.publish(state)
    rate.sleep()

In [9]:
####### RESET MODEL ########
T_state = human.get_initial_state()
human.send_state(T_state)

In [26]:
####### CLASS STATE TEST #######
class_value = 1
states = postures_data[class_value - 1]
for s in states:
    state = dicttostate(s).joint_state
    human.send_state(state)
    pub.publish(state)   
    udp_send_state(state)

    rate.sleep()

In [11]:
def send_joint(joint, value, port=5005):
    vect = np.zeros(92)    
    vect[joint] = value

    string_pattern = ('f ' * 92)[:-1]
    packer = struct.Struct(string_pattern)
    packed_data = packer.pack(*vect)

    sock = socket.socket(socket.AF_INET, # Internet
                         socket.SOCK_DGRAM) # UDP
    sock.sendto(packed_data, ("BAXTERFLOWERS.local", port))

In [19]:
def udp_send_state(joint_state, port=5005):
    # create a vector of 92 elements
    vect = np.zeros(92)
    vect[0] = joint_state.position[joint_state.name.index('spine_1')]
    vect[1] = joint_state.position[joint_state.name.index('spine_0')]
    vect[2] = joint_state.position[joint_state.name.index('spine_2')]
    
    vect[6] = joint_state.position[joint_state.name.index('neck_1')]
    vect[7] = joint_state.position[joint_state.name.index('neck_0')]
    vect[8] = joint_state.position[joint_state.name.index('neck_2')]
    
    vect[21] = joint_state.position[joint_state.name.index('left_knee_0')]
    vect[29] = joint_state.position[joint_state.name.index('right_knee_0')]
    
    vect[36] = -joint_state.position[joint_state.name.index('left_shoulder_0')]
    vect[37] = joint_state.position[joint_state.name.index('left_shoulder_1')]
    if 'left_shoulder_2' in joint_state.name:
        vect[38] = -joint_state.position[joint_state.name.index('left_shoulder_2')]
    
    vect[39] = joint_state.position[joint_state.name.index('left_elbow_0')]
    vect[40] = -joint_state.position[joint_state.name.index('left_wrist_2')]
    vect[41] = -joint_state.position[joint_state.name.index('left_wrist_1')]
    vect[42] = joint_state.position[joint_state.name.index('left_wrist_0')]
    
    vect[45] = -joint_state.position[joint_state.name.index('right_shoulder_0')]
    vect[46] = -joint_state.position[joint_state.name.index('right_shoulder_1')]
    if 'right_shoulder_2' in joint_state.name:
        vect[47] = joint_state.position[joint_state.name.index('right_shoulder_2')]
    
    vect[48] = -joint_state.position[joint_state.name.index('right_elbow_0')]
    vect[49] = -joint_state.position[joint_state.name.index('right_wrist_2')]
    vect[50] = -joint_state.position[joint_state.name.index('right_wrist_1')]
    vect[51] = joint_state.position[joint_state.name.index('right_wrist_0')]
    
    # offset at T pose
    vect[18] += 0.6
    vect[21] += 1.0
    vect[26] += 0.6
    vect[29] += 1.0
    vect[36] += 0.4
    vect[37] += 0.3
    vect[39] += 1.0
    vect[45] += 0.4
    vect[46] += 0.3
    vect[48] += 1.0
    
    vect[52] += -1.33
    vect[53] += -0.29
    vect[54] += 0.64
    vect[55] += 0.64
    vect[56] += 0.67
    vect[57] += -0.46
    vect[58] += 0.81
    vect[59] += 0.81
    vect[60] += 0.67
    vect[61] += -0.61
    vect[62] += 0.81
    vect[63] += 0.81
    vect[64] += 0.67
    vect[65] += -0.61
    vect[66] += 0.81
    vect[67] += 0.81
    vect[68] += 0.67
    vect[69] += -0.46
    vect[70] += 0.81
    vect[71] += 0.81
    vect[72] += -1.33
    vect[73] += -0.29
    vect[74] += 0.64
    vect[75] += 0.64
    vect[76] += 0.67
    vect[77] += -0.46
    vect[78] += 0.81
    vect[79] += 0.81
    vect[80] += 0.67
    vect[81] += -0.61
    vect[82] += 0.81
    vect[83] += 0.81
    vect[84] += 0.67
    vect[85] += -0.61
    vect[86] += 0.81
    vect[87] += 0.81
    vect[88] += 0.67
    vect[89] += -0.46
    vect[90] += 0.81
    vect[91] += 0.81
    
    string_pattern = ('f ' * 92)[:-1]
    packer = struct.Struct(string_pattern)
    packed_data = packer.pack(*vect)

    sock = socket.socket(socket.AF_INET, # Internet
                         socket.SOCK_DGRAM) # UDP
    sock.sendto(packed_data, ("BAXTERFLOWERS.local", port))

In [21]:
human.send_state(state)
pub.publish(state)
udp_send_state(state)

In [22]:
udp_send_state(T_state)

In [ ]:
state

In [ ]:
UDP_IP = "BAXTERFLOWERS.local"
UDP_PORT = 5005
MESSAGE = "Hello, World!"

print "UDP target IP:", UDP_IP
print "UDP target port:", UDP_PORT
print "message:", MESSAGE

sock = socket.socket(socket.AF_INET, # Internet
                     socket.SOCK_DGRAM) # UDP
sock.sendto(MESSAGE, (UDP_IP, UDP_PORT))

In [ ]:
for i in range(92):
    send_joint(i, 1)
    rospy.sleep(2)
    print i

In [ ]:
send_joint(43, 1)